In [1]:
import ollama
from IPython.display import Markdown, display, update_display

In [2]:
import datetime
import time

response_one = 0
response_two = 0
# Define the log file
log_file = "conversation-sample2.txt"

# Function to write and display logs with colors


def log_response(role, response):
    # Define colors
    colors = {
        "assistant": "\033[1;34m",  # Bright Blue
        "user": "\033[1;32m",  # Bright Green
        "timestamp": "\033[0;37m",  # Light Gray
        "reset": "\033[0m",  # Reset to default
    }
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Format for logging and console
    formatted_log = f"[{colors['timestamp']}{timestamp}{colors['reset']}] {colors[role.lower()]}{role.upper()}{colors['reset']}: {response}\n"

    # Print the colored log to the console
    # print(formatted_log, end="")

    # Save the log without color to the file
    with open(log_file, "a") as f:
        f.write(f"[{timestamp}] {role.upper()}: {response}\n")


# for _ in range(1):
def start_adversarial_game(
    messages_one,
    messages_two,
    model_one_system_prompt,
    model_two_system_prompt,
    MODEL_ONE,
    MODEL_TWO,
):
    # Generate response from MODEL_ONE
    response_one = ollama.chat(
        model=MODEL_ONE,
        messages=[{"role": "system", "content": model_one_system_prompt}] + messages_one,
    )["message"]["content"]
    messages_one.append({"role": "assistant", "content": response_one})
    messages_two.append({"role": "user", "content": response_one})
    log_response("assistant", response_one)

    response_two = ollama.chat(
        model=MODEL_TWO,
        messages=[{"role": "system", "content": model_two_system_prompt}] + messages_two,
    )["message"]["content"]
    messages_two.append({"role": "assistant", "content": response_two})
    messages_one.append({"role": "user", "content": response_two})
    log_response("user", response_two)

    # Trim the message history if it exceeds the limit
    if len(messages_one) > MAX_HISTORY:
        messages = messages_one[-MAX_HISTORY:]
    if len(messages_two) > MAX_HISTORY:
        messages = messages_two[-MAX_HISTORY:]

In [3]:
messages_one = []
messages_two = []
model_one_system_prompt = "You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way. Keep your conversation short."
model_two_system_prompt = "You are a very polite, courteous chatbot. You try to agree with everything the other person says, or find common ground. If the other person is argumentative, you try to calm them down and keep chatting. Keep your conversation short."

MODEL_TWO = "llama3:8b"
MODEL_ONE = "llama3.1:latest"

MAX_HISTORY = 7
for _ in range(50):
    start_adversarial_game(
        messages_one,
        messages_two,
        model_one_system_prompt,
        model_two_system_prompt,
        MODEL_ONE,
        MODEL_TWO,
    )

In [11]:
# Switching between models is very slow?
# https://github.com/ollama/ollama/issues/3115

In [1]:
with open("conversation-sample1.txt", "r") as file:
    content = file.read()

In [2]:
import re

# Function to clean the text


def clean_text(text):
    # Remove timestamps
    text = re.sub(r"\[\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\] ", "", text)

    # Remove labels (ASSISTANT, USER, etc.)
    text = re.sub(r"\b(ASSISTANT|USER):\s?", "", text)

    # Remove special characters except periods
    text = re.sub(r"[^\w\s.]", "", text)

    return text.strip()


# Function to parse logs and return a list of dialogues


def parse_logs(log_data):
    # Split the log by newlines to separate each line
    log_lines = log_data.split("\n")

    # Parse each line and clean it
    dialogues = [clean_text(line) for line in log_lines if line.strip()]

    return dialogues


# Parse the log data
dialogues = parse_logs(content)

In [3]:
import ChatTTS
import numpy as np
import torch
from IPython.display import Audio

# Initialize the ChatTTS model
chat = ChatTTS.Chat()
chat.load(compile=True, source="local", custom_path="/home/pranav-pc/projects/angel/")
# Generate speech from the text

True

In [4]:
###################################
# Sample a speaker from Gaussian.

rand_spk = chat.sample_random_speaker()


params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_emb=rand_spk,  # add sampled speaker
    temperature=0.3,  # using custom temperature
    top_P=0.7,  # top P decode
    top_K=20,  # top K decode
)

###################################
# For sentence level manual control.

# use oral_(0-9), laugh_(0-2), break_(0-7)
# to generate special token in text to synthesize.
params_refine_text = ChatTTS.Chat.RefineTextParams(
    prompt="[oral_2][laugh_0][break_6]",
)

wavs = chat.infer(
    dialogues,
    params_refine_text=params_refine_text,
    params_infer_code=params_infer_code,
)

# Function to trim silence (values close to zero) from the end of audio


def trim_silence(audio, threshold=1e-3):
    """
    Trims silence from the end of the audio.
    Args:
        audio (numpy array): The audio data.
        threshold (float): Values below this threshold are considered silence.
    Returns:
        numpy array: Trimmed audio.
    """
    non_silent_indices = np.where(np.abs(audio) > threshold)[0]
    if len(non_silent_indices) == 0:  # If all is silence
        return audio
    return audio[: non_silent_indices[-1] + 1]


# Trim silence from the end of each audio segment
trimmed_wavs = [trim_silence(wav) for wav in wavs]

# Merge all the audio into one tensor (along dimension 0 for 1D tensors)
merged_audio = torch.cat([torch.from_numpy(wav).squeeze() for wav in trimmed_wavs], dim=0)

# Play the merged audio directly in the notebook
display(Audio(merged_audio.numpy(), rate=24000))  # You can adjust the rate if needed

"""
In some versions of torchaudio, the first line works but in other versions, so does the second line.
"""
try:
    torchaudio.save("word_level_output.wav", torch.from_numpy(wavs[0]).unsqueeze(0), 24000)
except:
    torchaudio.save("word_level_output.wav", torch.from_numpy(wavs[0]), 24000)

text:   0%|                                      | 1/384(max) [00:08,  8.51s/it]We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
text:  35%|████████████▊                       | 136/384(max) [00:38, 13.22it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 124.00 MiB. GPU 0 has a total capacity of 11.72 GiB of which 92.56 MiB is free. Including non-PyTorch memory, this process has 11.36 GiB memory in use. Of the allocated memory 10.69 GiB is allocated by PyTorch, and 454.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

text:  36%|████████████▊                       | 137/384(max) [00:50, 13.22it/s]